In [42]:
import json, ndjson, pprint
from dotenv import load_dotenv

import openai

from time import sleep
import logging
import datetime

In [43]:
import os, sys
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

In [44]:
# APIキーをファイルから読み込み，APIキー認証
load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

In [45]:
# 処理時間を計測するため、loggerのフォーマット設定しています
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
format = '%(asctime)s [%(levelname)s] %(filename)s, lines %(lineno)d. %(message)s'
formatter = logging.Formatter(format, '%Y-%m-%d %H:%M:%S')
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [46]:
# トークログ(プロンプト)のバックアップログを取る関数
def AddBackUpLog_gptPrompt(prompt:dict):
    dt_now = datetime.datetime.now()
    BACKUP_LOG_PATH = f"./log/openai.ChatCompletion.create_{dt_now.year}-{dt_now.month}-{dt_now.day}.ndjson"
    with open(BACKUP_LOG_PATH, 'a') as f:
        writer = ndjson.writer(f, ensure_ascii=False)
        writer.writerow(prompt)

In [47]:
# リトライ付きChatCompletion
def ChatCompletion(
        messages:list, 
        model:str = "gpt-4", 
        max_token:int = 4000
):
    for message in messages:
        AddBackUpLog_gptPrompt(message) # GPT入力/出力のログを取る
    retry_interval = 2
    tries = 4
    for num_try in range(1, tries+1):
        try:
            print(f"START ChatCompletion.create Contents ... ", end="")
            response = openai.ChatCompletion.create(
                model = model,
                messages = messages,
                max_tokens = max_token
            )
            print(f"FINISH ChatCompletion.create")
            AddBackUpLog_gptPrompt(response["choices"][0]["message"]) # GPT入力/出力のログを取る
            break
        except:
            print(f"ChatCompletion.create ERROR")
            if num_try == tries:
                raise
            sleep(retry_interval)
            logger.info(f'リトライ回数:{num_try}回目')
            continue
    return response

In [48]:
QUESTION_INPUT_PATH = "../datas/question_sentence/dataset_02.json"

In [49]:
with open(QUESTION_INPUT_PATH, "r") as f:
    question_list = json.load(f)

pprint.pprint(question_list, width=150)

[{'URL': 'https://hokkaimath.jp/blog-entry-186.html',
  'id': 1,
  'impression': ['垂線がなかったけど，PerpendicularLine関数の存在を教えたらいけた．'],
  'normalized': '立体ABCD-EFGHは，AB=40cm，AD=30cm，AE=50cmの直方体である．頂点Dと頂点Fを結び，頂点Bから線分DFに引いた垂線と線分DFとの交点をIとする．線分BIの長さは何cmか．',
  'original': '立体ABCD-EFGHは，AB=40cm，AD=30cm，AE=50cmの直方体である．頂点Dと頂点Fを結び，頂点Bから線分DFに引いた垂線と線分DFとの交点をIとする．線分BIの長さは何cmか．',
  'script': ['A = (0, 0, 0),B = (40, 0, 0),C = (40, 30, 0),D = (0, 30, 0),E = (0, 0, 50),F = (40, 0, 50),G = (40, 30, 50),H = (0, 30, 50),df = '
             'Segment(D, F),Segment(A, B),Segment(B, C),Segment(C, D),Segment(D, A),Segment(E, F),Segment(F, G),Segment(G, H),Segment(H, '
             'E),Segment(A, E),Segment(B, F),Segment(C, G),Segment(D, H),perpendicularBI = PerpendicularLine(B, df),I = Intersect(perpendicularBI, '
             'df),']},
 {'URL': 'https://school.js88.com/scl_h/22007050/kakoq?kakomon_id=42300&img_type=1&page=4',
  'id': 2,
  'impression': ['四面体が描画できていない(長さ→座標が計算できてない)．'],
  'normalized': 'AB = √3，AC=A

In [50]:
system_prompt = """I enter a math problem statement in Japanese. Please, generate a GeoGebra Script to visualize it. 
Never abide by the following 2 rules:
1. if the exact coordinates are to be determined from the length, this can be done by solving the following equations.
Example
A(Xa,Ya),B(Xb,Yb),a=BC,b=AC, C = (x, y)
Then
(x-Xa)^2+(y-Ya)^2 = a^2
(x-Xb)^2+(y-Yb)^2 = b^2
2. Please do not leave any comments, return only geogebra script in code space.

Also,The following functions must also be used for specific Japanese words.
"線分": Segment()
"中点": Midpoint(Object, Object)
"垂線": PerpendicularLine()
"角形": Polygon()
"円錐": Cone(Circle, Height)
"円柱": Cylinder(Circle, Height)
"交点": Intersect(Object, Object)
"1回転": Surface(Object, 360, Line)"""

ex_q1 = "立体ABCD-EFGHは，AB=40cm，AD=30cm，AE=50cmの直方体である．頂点Dと頂点Fを結び，頂点Bから線分DFに引いた垂線と線分DFとの交点をIとする．線分BIの長さは何cmか．"
ex_a1 = """A = (0, 0, 0)
B = (40, 0, 0)
C = (40, 30, 0)
D = (0, 30, 0)
E = (0, 0, 50)
F = (40, 0, 50)
G = (40, 30, 50)
H = (0, 30, 50)
Segment(A, B)
Segment(B, C)
Segment(C, D)
Segment(D, A)
Segment(E, F)
Segment(F, G)
Segment(G, H)
Segment(H, E)
Segment(A, E)
Segment(B, F)
Segment(C, G)
Segment(D, H)
Segment(D, F)
I = PerpendicularLine(B, Segment(D, F))"""
ex_q2 = "AB=5 cm，BC=6 cm，AD=3 cm，∠C=∠D=90°の台形 ABCD を，辺 DC を軸として 1回転させてできる立体"
ex_a2 = """C = (0,0,0)
B = (6,0,0)
A = (3,0,4)
D = (0,0,4)
a = Segment(A, B)
b = Segment(B, C)
c = Segment(C, D)
d = Segment(D, A)
q1 = Polygon(A, B, C, D)
e = Surface(q1, 360, c)"""

In [51]:
messages = [
    {"role":"system", "content":system_prompt},
    {"role":"user", "content":ex_q1},
    {"role":"assistant", "content":ex_a1},
    {"role":"user", "content":ex_q2},
    {"role":"assistant", "content":ex_a2},
]

input_question = question_list[2]["normalized"]
messages.append({"role":"user", "content":input_question})

res = ChatCompletion(messages, 
                    #  model = "gpt-3.5-turbo-16k",
                     model = "gpt-4",
                     max_token = 4000)
pprint.pprint(res)

START ChatCompletion.create Contents ... FINISH ChatCompletion.create
{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'message': {'content': 'A = (0, 0, 0)\n'
                                     'B = (3, 0, 0)\n'
                                     'C = (1, sqrt(12), 0)\n'
                                     'D = (1, sqrt(12)/3, sqrt(20)/3)\n'
                                     'Segment(A, B)\n'
                                     'Segment(B, C)\n'
                                     'Segment(C, A)\n'
                                     'Segment(A, D)\n'
                                     'Segment(B, D)\n'
                                     'Segment(C, D)\n'
                                     'H = Midpoint(D, Polygon(A,B,C))\n'
                                     'PerpendicularLine(D, Polygon(A,B,C))',
                          'role': 'assistant'}}],
 'created': 1693292185,
 'id': 'chatcmpl-7smwDXrJlBCuy4Ey1apJgeA4cPTgf',
 'model': 'gpt-4-